In [112]:
import psycopg2
import gensim
from collections import namedtuple, Counter

model = gensim.models.doc2vec.Doc2Vec.load('../models/doc2vec_2k_primary_key')

with psycopg2.connect(dbname='arxiv') as conn:
    with conn.cursor() as cur:
        print(cur.itersize)
        cur.execute("SELECT * FROM articles limit 5")        
        col_names = [col.name for col in cur.description]
        Article = namedtuple("Article", col_names)
        hits = 0
        for row in cur:
            article = Article(*row)
            relateds = model.docvecs.most_similar(positive=[article.index], topn=2)
            related_ids = tuple([idx for idx, similarity in relateds])
            with conn.cursor() as newcur:
                newcur.execute("SELECT subject FROM articles WHERE index in %s"%(related_ids, ))
                cats = newcur.fetchall()
                new_hits = [1 if cat==article.subject else 0 for cat in cats]
                hits += sum(new_hits)

print(hits)
cur.arraysize

2000
0


1

In [113]:
with psycopg2.connect(dbname='arxiv') as conn:
    with conn.cursor() as cur:
        cur.execute("select count(*) from articles")
        result = cur.fetchall()
        print(result)

[(2000,)]


(Column(name='index', type_code=23, display_size=None, internal_size=4, precision=None, scale=None, null_ok=None),
 Column(name='title', type_code=25, display_size=None, internal_size=-1, precision=None, scale=None, null_ok=None),
 Column(name='authors', type_code=25, display_size=None, internal_size=-1, precision=None, scale=None, null_ok=None),
 Column(name='subject', type_code=25, display_size=None, internal_size=-1, precision=None, scale=None, null_ok=None),
 Column(name='abstract', type_code=25, display_size=None, internal_size=-1, precision=None, scale=None, null_ok=None),
 Column(name='last_submitted', type_code=1082, display_size=None, internal_size=4, precision=None, scale=None, null_ok=None),
 Column(name='arxiv_id', type_code=25, display_size=None, internal_size=-1, precision=None, scale=None, null_ok=None))